In [ ]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import plotly.express as px
import numpy as np

def train_and_evaluate_prophet_model_1(df, date_column, target_column, input_month):
    # Preprocess the dataset
    df = df.dropna(thresh=len(df) * 0.25, axis=1)
    df[date_column] = pd.to_datetime(df[date_column])
    df = df.sort_values(by=date_column)
    
    # Filter the data for the input month
    df['month'] = df[date_column].dt.month
    df_month = df[df['month'] == input_month]

    # Determine the year to use for testing
    max_year = df_month[date_column].dt.year.max()
    if df_month[df_month[date_column].dt.year == max_year].empty:
        test_year = max_year - 1
    else:
        test_year = max_year

    # Split into training and test sets
    train = df_month[df_month[date_column].dt.year < test_year]
    test = df_month[df_month[date_column].dt.year == test_year]

    # Identify common columns (regressors) in both train and test sets
    common_columns = set(train.columns) & set(test.columns)
    regressor_columns = list(common_columns - {date_column, target_column, 'month'})

    # Prepare the data for Prophet
    train_prophet = train[[date_column, target_column] + regressor_columns].rename(columns={date_column: 'ds', target_column: 'y'})
    test_prophet = test[[date_column, target_column] + regressor_columns].rename(columns={date_column: 'ds', target_column: 'y'})

    # Initialize and fit the model
    model = Prophet()
    for column in regressor_columns:
        model.add_regressor(column)
    model.fit(train_prophet)

    # Make predictions and calculate errors
    forecast = model.predict(test_prophet.drop(columns='y'))
    forecast = forecast[['ds', 'yhat']]
    test_merged = test_prophet.merge(forecast, on='ds')
    test_merged['error'] = test_merged['y'] - test_merged['yhat']
    average_error = np.mean(abs(test_merged['error']))

    # Identify days with drastically increased error
    significant_error_days = test_merged[test_merged['error'].abs() > average_error * 2]

    mse = mean_squared_error(test_merged['y'], test_merged['yhat'])
    mae = mean_absolute_error(test_merged['y'], test_merged['yhat'])

    # Create a Plotly chart showing daily error
    fig = px.bar(test_merged, x='ds', y='error', title='Daily Error in Predictions')

    # Analyze the impact of regressors
    regressor_importance = model.params['beta'][0][:len(regressor_columns)]
    total_importance = sum(abs(regressor_importance))
    importance_df = pd.DataFrame({
        'Regressor': regressor_columns,
        'Importance': abs(regressor_importance),
        'Relative Importance (%)': 100 * abs(regressor_importance) / total_importance
    })
    importance_df = importance_df.sort_values(by='Relative Importance (%)', ascending=False)

    return mse, mae, fig, importance_df, significant_error_days

# Example usage:
# mse, mae, fig, importance_df, significant_error_days = train_and_evaluate_prophet_model(your_dataframe, 'your_date_column', 'your_target_column', your_input_month)
# if fig is not None:
#     fig.show()
# if not significant_error_days.empty:
#     print(significant_error_days)

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

# Example usage of your existing function
# trained_model, mse, mae, fig, importance_df, significant_error_days, regressor_columns = train_and_evaluate_prophet_model(your_dataframe, 'your_date_column', 'your_target_column', your_input_month)

# Create input widgets for each regressor
input_widgets = [widgets.Text(description=regressor, value='') for regressor in regressor_columns]
button = widgets.Button(description="Predict")
output = widgets.Output()

display(*input_widgets, button, output)

def on_button_clicked(b):
    with output:
        output.clear_output()
        # Collect input values
        regressor_values = {widget.description: float(widget.value) for widget in input_widgets}
        prediction_df = pd.DataFrame([regressor_values])

        # Add a dummy date column for prediction
        prediction_df['ds'] = pd.to_datetime("2023-01-01")  # Use an arbitrary date

        # Make prediction using the trained model
        forecast = trained_model.predict(prediction_df)
        predicted_value = forecast['yhat'].iloc[0]
        print(f"Predicted Target Value: {predicted_value}")

button.on_click(on_button_clicked)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd

# Assuming regressor_columns is defined and contains the names of your regressors
# Example: regressor_columns = ['long_regressor_name1', 'long_regressor_name2', ...]

# Create input widgets for each regressor without a description
input_widgets = [widgets.Text(value='', placeholder=regressor) for regressor in regressor_columns]

# Create the prediction button and output display
button = widgets.Button(description="Predict")
output = widgets.Output()

# Display widgets with HTML labels
for reg, widget in zip(regressor_columns, input_widgets):
    display(HTML(f'<label style="width: 100%; text-align: left;">{reg}</label>'))
    display(widget)

display(button, output)

def on_button_clicked(b):
    with output:
        output.clear_output()
        # Collect input values
        regressor_values = {widget.placeholder: float(widget.value) for widget in input_widgets}
        prediction_df = pd.DataFrame([regressor_values])

        # Add a dummy date column for prediction
        prediction_df['ds'] = pd.to_datetime("2023-01-01")  # Use an arbitrary date

        # Make prediction using the trained model
        forecast = trained_model.predict(prediction_df)
        predicted_value = forecast['yhat'].iloc[0]
        print(f"Predicted Target Value: {predicted_value}")

button.on_click(on_button_clicked)


In [ ]:
import pandas as pd
import datetime
import requests

# Function to get today's date in the required format
def get_todays_date():
    return datetime.datetime.now().strftime('%Y-%m-%d')

# Your base URL without the end date
base_url = "http://commod_s3_url_with_parameters_up_to_end_date="

# Today's date
today = get_todays_date()

# Complete URL with today's date as the end date
url_with_todays_date = f"{base_url}{today}"

# Fetch the data
response = requests.get(url_with_todays_date)
if response.status_code == 200:
    # Assuming the data is in CSV format
    df = pd.read_csv(url_with_todays_date)
    # Save the DataFrame to a CSV file, for example 'daily_data.csv'
    df.to_csv('daily_data.csv', index=False)
else:
    print("Failed to fetch data:", response.status_code)


In [ ]:
def get_10_day_averages(df, date_column, target_date, regressor_columns):
    """
    Calculate the average of each regressor for the 10 days leading up to and including the target date.

    Parameters:
    df (DataFrame): The DataFrame containing the historical data.
    date_column (str): The name of the column in df that contains the dates.
    target_date (str): The date for which to calculate the averages, in 'YYYY-MM-DD' format.
    regressor_columns (list): A list of column names to calculate the averages for.

    Returns:
    dict: A dictionary with the regressor names as keys and their 10-day averages as values.
    """

    # Convert date_column to datetime and ensure target_date is a datetime object
    df[date_column] = pd.to_datetime(df[date_column])
    target_date = pd.to_datetime(target_date)

    # Define the 10 day period
    start_date = target_date - pd.Timedelta(days=9)

    # Filter the DataFrame for the 10 day period
    mask = (df[date_column] >= start_date) & (df[date_column] <= target_date)
    filtered_df = df.loc[mask]

    # Calculate the mean for each regressor
    means = filtered_df[regressor_columns].mean()

    return means.to_dict()

# Example usage:
# averages = get_10_day_averages(your_dataframe, 'your_date_column', '2023-01-01', ['regressor1', 'regressor2'])
# print(averages)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd

# Assuming you have a DataFrame 'df' and a list of regressor columns 'regressor_columns'
# regressor_columns = ['regressor1', 'regressor2', ...]

# Get the 10-day averages for the regressors
averages = get_recent_10_day_averages(df, 'date_column', regressor_columns)

# Create input widgets for each regressor with the average values as default
input_widgets = [widgets.Text(value=str(averages.get(regressor, '')), placeholder=regressor, description=regressor) for regressor in regressor_columns]

# Create the prediction button and output display
button = widgets.Button(description="Predict")
output = widgets.Output()

for widget in input_widgets:
    display(widget)

display(button, output)

def on_button_clicked(b):
    with output:
        output.clear_output()
        # Collect input values
        regressor_values = {widget.description: float(widget.value) for widget in input_widgets}
        prediction_df = pd.DataFrame([regressor_values])
        prediction_df['ds'] = pd.to_datetime("2023-01-01")  # Use an arbitrary date

        # Make prediction using the trained model
        forecast = trained_model.predict(prediction_df)
        predicted_value = forecast['yhat'].iloc[0]
        print(f"Predicted Target Value: {predicted_value}")

button.on_click(on_button_clicked)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd

# Assuming you have a DataFrame 'df' and a list of regressor columns 'regressor_columns'
# regressor_columns = ['regressor1', 'regressor2', ...]

# Get the 10-day averages for the regressors
averages = get_recent_10_day_averages(df, 'date_column', regressor_columns)

# Set a layout for the widgets with a wider description field
description_layout = widgets.Layout(width='150px')  # Adjust the width as needed

# Create input widgets for each regressor with the average values as default
input_widgets = [widgets.Text(value=str(averages.get(regressor, '')), 
                              placeholder=regressor, 
                              description=regressor,
                              layout=description_layout) for regressor in regressor_columns]

# Create the prediction button and output display
button = widgets.Button(description="Predict")
output = widgets.Output()

for widget in input_widgets:
    display(widget)

display(button, output)

def on_button_clicked(b):
    with output:
        output.clear_output()
        # Collect input values
        regressor_values = {widget.description: float(widget.value) for widget in input_widgets}
        prediction_df = pd.DataFrame([regressor_values])
        prediction_df['ds'] = pd.to_datetime("2023-01-01")  # Use an arbitrary date

        # Make prediction using the trained model
        forecast = trained_model.predict(prediction_df)
        predicted_value = forecast['yhat'].iloc[0]
        print(f"Predicted Target Value: {predicted_value}")

button.on_click(on_button_clicked)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd

# Assuming you have a DataFrame 'df' and a list of regressor columns 'regressor_columns'
# regressor_columns = ['regressor1', 'regressor2', ...]

# Get the 10-day averages for the regressors
averages = get_recent_10_day_averages(df, 'date_column', regressor_columns)

# Custom style to increase description width
style = {'description_width': 'initial'}  # Adjust as needed

# Create input widgets for each regressor with the average values as default
input_widgets = [widgets.Text(value=str(averages.get(regressor, '')), 
                              placeholder=regressor, 
                              description=regressor,
                              style=style) for regressor in regressor_columns]

# Create the prediction button and output display
button = widgets.Button(description="Predict")
output = widgets.Output()

for widget in input_widgets:
    display(widget)

display(button, output)

def on_button_clicked(b):
    with output:
        output.clear_output()
        # Collect input values
        regressor_values = {widget.description: float(widget.value) for widget in input_widgets}
        prediction_df = pd.DataFrame([regressor_values])
        prediction_df['ds'] = pd.to_datetime("2023-01-01")  # Use an arbitrary date

        # Make prediction using the trained model
        forecast = trained_model.predict(prediction_df)
        predicted_value = forecast['yhat'].iloc[0]
        print(f"Predicted Target Value: {predicted_value}")

button.on_click(on_button_clicked)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd

# Assuming you have the importance_df DataFrame with columns 'Regressor' and 'Relative Importance (%)'
# And assuming you have a DataFrame 'df' and a list of regressor columns 'regressor_columns'
# regressor_columns = ['regressor1', 'regressor2', ...]

# Get the 10-day averages for the regressors
averages = get_recent_10_day_averages(df, 'date_column', regressor_columns)

# Create a dictionary for relative importance from importance_df
relative_importance = importance_df.set_index('Regressor')['Relative Importance (%)'].to_dict()

# Define the width for the input boxes
input_box_layout = widgets.Layout(width='200px')

# Create input widgets and HTML labels
input_widgets = []
for regressor in regressor_columns:
    # Create HTML label with regressor name and relative importance
    importance = f" ({relative_importance.get(regressor, 0):.2f}%)" if regressor in relative_importance else ""
    label_html = HTML(f'<label style="width: 100%; text-align: left;">{regressor}{importance}</label>')
    
    # Create Text widget for input
    text_widget = widgets.Text(value=str(averages.get(regressor, '')), 
                               placeholder=regressor, 
                               layout=input_box_layout)

    input_widgets.append((label_html, text_widget))

# Create the prediction button and output display
button = widgets.Button(description="Predict")
output = widgets.Output()

# Display widgets
for label, widget in input_widgets:
    display(label, widget)

display(button, output)

def on_button_clicked(b):
    with output:
        output.clear_output()
        # Collect input values
        regressor_values = {widget.placeholder: float(widget.value) for _, widget in input_widgets}
        prediction_df = pd.DataFrame([regressor_values])
        prediction_df['ds'] = pd.to_datetime("2023-01-01")  # Use an arbitrary date

        # Make prediction using the trained model
        forecast = trained_model.predict(prediction_df)
        predicted_value = forecast['yhat'].iloc[0]
        print(f"Predicted Target Value: {predicted_value}")

button.on_click(on_button_clicked)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd

# Assuming you have the importance_df DataFrame with columns 'Regressor' and 'Relative Importance (%)'
# And assuming you have a DataFrame 'df' and a list of regressor columns 'regressor_columns'
# regressor_columns = ['regressor1', 'regressor2', ...]

# Sort the regressor_columns based on relative importance
sorted_regressors = importance_df.sort_values(by='Relative Importance (%)', ascending=False)['Regressor']
sorted_regressors = [reg for reg in sorted_regressors if reg in regressor_columns]

# Get the 10-day averages for the regressors
averages = get_recent_10_day_averages(df, 'date_column', sorted_regressors)

# Define the width for the input boxes
input_box_layout = widgets.Layout(width='200px')

# Create input widgets and HTML labels
input_widgets = []
for regressor in sorted_regressors:
    importance = f" ({importance_df.set_index('Regressor').loc[regressor, 'Relative Importance (%)']:.2f}%)"
    label_html = HTML(f'<label style="width: 100%; text-align: left;">{regressor}{importance}</label>')
    text_widget = widgets.Text(value=str(averages.get(regressor, '')), 
                               placeholder=regressor, 
                               layout=input_box_layout)

    input_widgets.append((label_html, text_widget))

# Create the prediction button and output display
button = widgets.Button(description="Predict")
output = widgets.Output()

# Display widgets
for label, widget in input_widgets:
    display(label, widget)

display(button, output)

def on_button_clicked(b):
    with output:
        output.clear_output()
        # Collect input values
        regressor_values = {widget.placeholder: float(widget.value) for _, widget in input_widgets}
        prediction_df = pd.DataFrame([regressor_values])
        prediction_df['ds'] = pd.to_datetime("2023-01-01")  # Use an arbitrary date

        # Make prediction using the trained model
        forecast = trained_model.predict(prediction_df)
        predicted_value = forecast['yhat'].iloc[0]
        print(f"Predicted Target Value: {predicted_value}")

button.on_click(on_button_clicked)
